# Fine-Tuning GPT chat bot for toshodex.


In [1]:
#%pip uninstall -y tensorflow
#%pip uninstall -y transformers

In [2]:
# Jupyter notebook cells
# Cell 1:
# Required packages
%pip install pandas mysql-connector-python
%pip install tensorflow
%pip install tensorflow-gpu
%pip install transformers
%pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [41 lines of output]
      Traceback (most recent call last):
        File "/home/tommy/.virtualenvs/toshodex-ipqa/lib/python3.10/site-packages/setuptools/_vendor/packaging/requirements.py", line 35, in __init__
          parsed = parse_requirement(requirement_strin

In [3]:
# Import necessary libraries
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer


2023-07-16 18:45:05.321802: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-16 18:45:05.323418: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 18:45:05.355835: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 18:45:05.356705: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-16 18:45:05.904764: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [5]:
# Cell 2:
# Importing necessary libraries
import pandas as pd
import mysql.connector

# Cell 3:
# Establish connection with the MySQL container
cnx = mysql.connector.connect(
    user='root', 
	# password='password',
	host='localhost',
	database='toshodex',
	port='3307'
)

# Cell 4:
# Query to fetch data from the wp_posts table
query = "SELECT post_title, post_excerpt, post_content FROM wp_posts where post_type='archive';"

# Cell 5:
# Create a cursor object, execute the query and fetch all rows into a pandas dataframe
cursor = cnx.cursor()
cursor.execute(query)

# Fetch rows into a list
rows = cursor.fetchall()

# Convert list into DataFrame
df = pd.DataFrame(rows, columns=[i[0] for i in cursor.description])

# Cell 6:
# Close the cursor and connection
cursor.close()
cnx.close()

# Cell 7:
# Print DataFrame
print(df)

# Cell 8:
# Preprocessing - concatenate title, excerpt, and content
df['text'] = df['post_title'] + " " + df['post_excerpt'] + " " + df['post_content']



                                           post_title post_excerpt  \
0                                      IWNC  Mongolia                
1                                         Suzu + Kizu                
2       MVP Spacial App (need to get link from Sembo)                
3                               Studio.pathfinder.com                
4                            Dining Out Concept Lexus                
5                                      Adult Learning                
6   Pioneered by Pathfinder 1 year before. The Fir...                
7                                          Life Patch                
8                                 Virtual Exhibitions                
9                             Digital Remote Learning                
10  Digital Literacy Quesiontaire for K-12 Student...                
11                                   Locus Of Control                
12                          Self-Esteem Self efficacy                
13                  

In [12]:
# check for write access
# result="Hello World"
# with open('result.txt', 'a') as fp:
#     fp.write(result)


In [15]:
# Cell 9:
# Load the pre-trained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# Add pad_token to tokenizer if not already there
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Cell 10:
# Prepare your data
texts = df['text'].tolist()  # Convert column to list
inputs = tokenizer(texts, return_tensors='tf', truncation=True, padding=True, max_length=256)

# We need to shift the input IDs to the right and also create labels for the model.
input_ids = inputs['input_ids'][:, :-1]
labels = inputs['input_ids'][:, 1:]
attention_mask = inputs['attention_mask'][:, :-1]

# Cell 11:
# Creating a tf.data.Dataset for training
BATCH_SIZE = 4
BUFFER_SIZE = 1000
# Create a dictionary for input tensors
data = {
    'input_ids': input_ids,
    'attention_mask': attention_mask,
    'labels': labels
}

dataset = tf.data.Dataset.from_tensor_slices(data)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Cell 12:
# Define a function for a custom training loop
def train_model(dataset, model, epochs):
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

    for epoch in range(epochs):
        print(f'Starting epoch {epoch+1}')
        
        for batch in dataset:
            with tf.GradientTape() as tape:
                outputs = model(batch)
                loss = outputs.loss
                
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        print(f'Loss after epoch {epoch+1}: {loss.numpy()}')

# Train the model
train_model(dataset, model, epochs=10)

# Cell 13:
# Save the fine-tuned model
model.save_pretrained("./saved_model2")


Using pad_token, but it is not set yet.
All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Starting epoch 1
Loss after epoch 1: [2.4832761]
Starting epoch 2
Loss after epoch 2: [1.4196534]
Starting epoch 3
Loss after epoch 3: [6.0338626]
Starting epoch 4
Loss after epoch 4: [2.5896893]
Starting epoch 5
Loss after epoch 5: [1.3967987]
Starting epoch 6
Loss after epoch 6: [3.044947]
Starting epoch 7
Loss after epoch 7: [3.420957]
Starting epoch 8
Loss after epoch 8: [2.3972914]
Starting epoch 9
Loss after epoch 9: [1.573567]
Starting epoch 10
Loss after epoch 10: [1.5585794]


In [14]:
%pwd
%ls saved_model2

config.json  generation_config.json  tf_model.h5
